In [1]:
## install environment
# !conda install --file ../env/byoml_custom.environment.yml
# !conda activate byoml_custom
# !pip install -r ../env/byoml_custom.requirements.txt

In this notebook we will demonstrate how to create your own simple custom models to upload to Waylay BYOML. We will create a function which doubles the inputs.
The dependencies in this environment are:
* waylay-beta
* dill

A best practice for serializing models is using `Python 3.7`, since the latest `dill` version is not yet up to date with newer versions of Python.

In [2]:
MODEL_NAME = 'test_custom_model'

In [3]:
import numpy as np

The following cell contains the custom model which has to contain a `predict` method. We would also like to map the predictions to our custom mapping function, all we have to do for this is create a method called `map_output`.

In [4]:
class Doubler:
    def __init__(self):
        self.keys = ['foo', 'bar', 'baz', 'something', 'else']
        
    def predict(self, data):
        return data * 2
    
    def map_output(self, request, predictions):
        mapped_predictions = [
            {
                self.keys[idx]: value
                for idx, value in enumerate(pred)
            }
            for pred in predictions.tolist()
        ]
        return {'predictions': mapped_predictions}

In [5]:
X = np.array([[1, 2, 3, 4, 5]])

In [6]:
doubler = Doubler()
preds = doubler.predict(X)
preds

array([[ 2,  4,  6,  8, 10]])

In [7]:
doubler.map_output({}, preds)

{'predictions': [{'foo': 2, 'bar': 4, 'baz': 6, 'something': 8, 'else': 10}]}

In [8]:
from waylay import WaylayClient, RestResponseError
waylay = WaylayClient.from_profile('staging')

In [9]:
waylay.byoml.model.upload(
        model_name=MODEL_NAME,
        trained_model=doubler,
        framework='custom',
        description='Simple custom model example'
    )

{'createdOn': '2021-01-04T00:00:00+00:00',
 'description': 'Simple custom model example',
 'framework': 'custom',
 'framework_version': '1.0',
 'message': 'Model successfully uploaded',
 'metadata': {'description': 'Simple custom model example'},
 'name': 'test_custom_model',
 'updatedOn': '2021-01-04T00:00:00+00:00',
 'version': '1'}

In [10]:
waylay.byoml.model.get(MODEL_NAME, params={'debug':'true'})

{'createdBy': 'users/f0981778-feed-4d08-823e-dc65e2b8a502',
 'createdOn': '2022-03-01T09:22:09.662000+00:00',
 'description': 'Simple custom model example',
 'framework': 'custom',
 'framework_version': '1.0',
 'metadata': {'description': 'Simple custom model example'},
 'name': 'test_custom_model',
 'ready': False,
 'runtime': {'framework': 'custom',
  'framework_version': '1.0',
  'name': 'byoml-custom-1.0',
  'python_version': '3.7'},
 'semantic_version': '0.0.1',
 'serving_name': 'kf-e8ca4239b5f7ed0bc831e473c0bf9f5811de6ec1.openfaas-fn-6ccc8843-d78d-49e8-84c4-3734a4af9929',
 'status': 'running',
 'supported_endpoints': [],
 'updatedBy': 'users/f0981778-feed-4d08-823e-dc65e2b8a502',
 'updatedOn': '2022-03-01T09:22:31.206000+00:00',
 'version': '1'}

In [11]:
X_test = [[1, 2, 3, 4, 5], [10, 20, 30, 40, 50]]

In [12]:
waylay.byoml.model.predict(MODEL_NAME, X_test)

array([{'foo': 2, 'bar': 4, 'baz': 6, 'something': 8, 'else': 10},
       {'foo': 20, 'bar': 40, 'baz': 60, 'something': 80, 'else': 100}],
      dtype=object)

In [13]:
waylay.byoml.model.remove(MODEL_NAME)

{'message': 'Model test_custom_model: all versions deleted',
 'name': 'test_custom_model',
 'versions': ['0.0.1']}